# 뉴스기사 분석
- 문서-단어 행렬의 예시 (Document-term Matrix)
- 빅카인즈 뉴스기사 데이터 활용(https://www.bigkinds.or.kr/): 한국언론진흥재단이 운영하는 사이트로
  국내 언론사들 뉴스기사의 분야별 키워드, 통계, 이슈 등을 제공한다.
- 여기서는 2019년 뉴스 데이터 사용

In [12]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [13]:
# 뉴스 데이터 가져오기
!curl -L https://bit.ly/2X7UON2 -o news.xlsx

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   162  100   162    0     0    152      0  0:00:01  0:00:01 --:--:--   152

  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0

  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0

100   990    0   990    0     0    406      0 --:--:--  0:00:02 --:--:--   406
100   990    0   990    0     0    406      0 --:--:--  0:00:02 --:--:--     0

 19 1789k   19  350k    0     0   109k      0  0:00:16  0:00:03  0:00:13  109k
100 1789k  100 1789k    0     0   489k      0  0:00:03  0:00:03 --:--:-- 3291k


In [14]:
# !dir

In [41]:
news_all = pd.read_excel("news.xlsx")
news_all.columns

Index(['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3',
       '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드', '특성추출',
       '본문', 'URL', '분석제외 여부'],
      dtype='object')

In [42]:
news_all.head(2)

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출,본문,URL,분석제외 여부
0,1.500401e+06,20190228,국제신문,김태경 기자 tgkim@kookje.co.kr,빅딜이냐 스몰딜이냐 ‘영변 핵+α’ 로드맵 도출이 관건,정치>북한,정치>외교,NaN,NaN,NaN,NaN,"김,도널드 트럼프,김의겸,김정은","북미,미국,하노이,서울,싱가포르,북한,리비아","북미,북미 정상회,미국,청와대,북미 정상회담,북한,국무위","빅딜,스몰딜,영변,로드맵,도출,비핵화,수준,상응,조치,막판,양국,협상팀,밀당,1차,...","북미,비핵화,미국,스몰딜,1차,정상회담,김정은,종전선언,연락사무소,하노이,영변,대변...",- 비핵화 수준 상응 조치 놓고\n- 양국 협상팀 막판까지 ‘밀당’\n- 1차 때와...,http://www.kookje.co.kr/news2011/asp/newsbody....,NaN
1,1.200201e+06,20190228,경인일보,조영상,"하노이 도착후 실무대표단 청취 김정은, 북미회담 성공 '잰걸음'",국제>국제일반,정치>북한,NaN,NaN,NaN,NaN,김,"북미,하노이,베트남","조선중앙통신,국무위","하노이,도착,실무대표단,청취,김정은,잰걸음,북미회담,성공,김정은,국무,위원장,시작,...","실무대표단,하노이,김정은,위원장,베트남,북미정상회담,북미회담,정상회담,조선중앙통신,...",김정은 국무위원장이 27일 시작되는 제2차 북미정상회담 성공을 위해 심혈을 기울이고...,http://www.kyeongin.com/main/view.php?key=2019...,NaN


In [43]:
news_text = news_all['본문']
news_text[:5]

0    - 비핵화 수준 상응 조치 놓고\n- 양국 협상팀 막판까지 ‘밀당’\n- 1차 때와...
1    김정은 국무위원장이 27일 시작되는 제2차 북미정상회담 성공을 위해 심혈을 기울이고...
2    북미가 처음으로 정상 간 단독회담과 만찬을 가지며 또다시 새로운 역사 창조에 나섰다...
3    지난해 9월 남북정상회담 당시 리선권 북한 조국평화통일위원장의 '냉면' 발언으로 정...
4    지자체 민간 교류 활성화 대부분 \n여, 부처간 논의 예산 지원 확대 \n야, 사업...
Name: 본문, dtype: object

In [44]:
type(news_text)

pandas.core.series.Series

In [91]:
cv = CountVectorizer()
dtm = cv.fit_transform(news_text)
df = pd.DataFrame(pd.DataFrame(dtm.toarray(), columns = cv.get_feature_names()))
df[:5]

,00,000원을,001420,001550,002100,005690,01,017800,02,025860,...,힌국당,힘겨루기로,힘겨웠던,힘들다고,힘들어,힘들어지는,힘을,힘이,힘입어,靈山
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
# 교재 내용
# cv = CountVectorizer()
# dtm = cv.fit_transform(news_text.tolist())
# df = pd.DataFrame(pd.DataFrame(dtm.toarray(), columns = cv.get_feature_names()))
# df.shape

In [93]:
df.shape    # 총 단어의 수가 16,1999 개

(1543, 16199)

In [94]:
# 어떤 단어들이 등장했는지 확인
cv.vocabulary_

{'비핵화': 7360,
 '수준': 8381,
 '상응': 7656,
 '조치': 12681,
 '놓고': 3926,
 '양국': 9295,
 '협상팀': 15748,
 '막판까지': 5281,
 '밀당': 6020,
 '1차': 209,
 '때와': 5047,
 '달리': 4149,
 '오랜': 9994,
 '시간': 8523,
 '조율': 12644,
 '단계적': 4098,
 '접근': 12173,
 '동의': 4794,
 '예상': 9918,
 '종전선언': 12736,
 '연락사무소': 9703,
 '개설': 1405,
 '조건부': 12565,
 '제재': 12534,
 '완화': 10125,
 '명시': 5596,
 '가능성': 1071,
 '북미회담': 7032,
 '빅딜만': 7392,
 '성공아냐': 8001,
 '김정은': 3223,
 '답방': 4194,
 '회담': 16011,
 '결과와': 1719,
 '연관': 9654,
 '북한': 7082,
 '국무위원장과': 2573,
 '도널드': 4592,
 '트럼프': 14545,
 '미국': 5886,
 '대통령이': 4500,
 '27일': 329,
 '친교': 14174,
 '만찬': 5368,
 '국무위원장이': 2582,
 '시작되는': 8626,
 '제2차': 12450,
 '북미정상회담': 7008,
 '성공을': 8005,
 '위해': 10523,
 '심혈을': 8906,
 '기울이고': 3041,
 '있는': 11487,
 '것으로': 1640,
 '나타났다': 3462,
 '조선중앙통신은': 12609,
 '이날': 10818,
 '26일': 320,
 '하노이에': 15095,
 '도착해': 4653,
 '멜리아': 5566,
 '호텔에서': 15812,
 '실무대표단의': 8792,
 '보고를': 6689,
 '받았다며': 6194,
 '조미': 12588,
 '수뇌회담의': 8297,
 '성공적': 8009,
 '보장을': 6799,
 '북미': 6986,
 '나라

In [95]:
# 문서-단어 행렬에서 가장 많이 등장하는 단어 찾기 (이것이 훨씬 간단)
df.sum()    # column-wise sum
pd.Series(df.sum()).idxmax()

'2차'

In [96]:
# 교재에 나타난 코드 (결과는 위와 동일)
top_word = []
for i in df.T.values:    # df.T.values 와 동일 (2-d array)
    top_word.append(sum(i.tolist()))
    
df.transpose().index[top_word.index(max(top_word))]

'2차'

In [107]:
# maximum 2000 개만 사용하도록 하기 위해서는?
cv = CountVectorizer(max_features=2000)
dtm = cv.fit_transform(news_text)
df = pd.DataFrame(pd.DataFrame(dtm.toarray(), columns = cv.get_feature_names()))
df[:6].T

,0,1,2,3,4,5
00,0,0,0,0,0,0
01,0,0,0,0,0,0
02,0,0,0,0,0,0
0px,0,0,0,0,0,0
10,0,0,0,0,0,0
100주년,0,0,0,0,0,0
100주년을,0,0,0,0,0,0
10시,0,0,0,0,0,0
10일,0,0,0,0,0,0
10일까지,0,0,0,0,0,0


In [111]:
df.T.max()

0       2
1       2
2       2
3       1
4       1
5       3
6       2
7       1
8       2
9       3
10      1
11      1
12      3
13      2
14      3
15      2
16      1
17      2
18      2
19      2
20      2
21      2
22      3
23      2
24      3
25      3
26      2
27      2
28      2
29      2
       ..
1513    2
1514    2
1515    4
1516    3
1517    4
1518    2
1519    1
1520    2
1521    1
1522    2
1523    2
1524    3
1525    2
1526    2
1527    2
1528    2
1529    2
1530    1
1531    2
1532    4
1533    1
1534    1
1535    1
1536    2
1537    1
1538    2
1539    2
1540    1
1541    3
1542    2
Length: 1543, dtype: int64

In [106]:
tv = TfidfVectorizer(max_features=2000)
dtm = tv.fit_transform(news_text.tolist())
df = pd.DataFrame(pd.DataFrame(dtm.toarray(), columns = tv.get_feature_names()))
df[:6].T

,0,1,2,3,4,5
00,0.000000,0.0,0.0,0.0,0.000000,0.0
01,0.000000,0.0,0.0,0.0,0.000000,0.0
02,0.000000,0.0,0.0,0.0,0.000000,0.0
0px,0.000000,0.0,0.0,0.0,0.000000,0.0
10,0.000000,0.0,0.0,0.0,0.000000,0.0
100주년,0.000000,0.0,0.0,0.0,0.000000,0.0
100주년을,0.000000,0.0,0.0,0.0,0.000000,0.0
10시,0.000000,0.0,0.0,0.0,0.000000,0.0
10일,0.000000,0.0,0.0,0.0,0.000000,0.0
10일까지,0.000000,0.0,0.0,0.0,0.000000,0.0


In [51]:
# 연습
df2 = pd.DataFrame(np.arange(12).reshape(3,4)); df2
df2.values

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [52]:
df2.sum()

0    12
1    15
2    18
3    21
dtype: int64

In [53]:
df2.max()

0     8
1     9
2    10
3    11
dtype: int32

In [60]:
df2.idxmax(axis=1)

0    3
1    3
2    3
dtype: int64

In [61]:
df2.T.idxmax()

0    3
1    3
2    3
dtype: int64

In [62]:
df2.sum()

0    12
1    15
2    18
3    21
dtype: int64

In [72]:
for i in df2.values:
    print(sum(i))

6
22
38


In [67]:
tmp=[]
for i in df2.T.values:
    tmp.append(sum(i))

In [68]:
tmp

[12, 15, 18, 21]

In [70]:
tmp=np.array(df2.sum()); tmp

array([12, 15, 18, 21], dtype=int64)

In [75]:
tmp_s = pd.Series(tmp)
tmp_s.idxmax()

3